Import the packages

In [1]:
import sys
# @Leila: Why are you inserting this path?
# sys.path.insert(0, '../../../')
import byxtal.lattice as gbl
import byxtal.integer_manipulations as iman
import byxtal.bp_basis as bpb
import numpy as np
import math as mt
import gbpy.util_funcs_create_byxtal as uf
import gbpy.generate_hkl_indices as ghi
from sympy.matrices import Matrix, eye, zeros

## Sympy + SAGE routines
import integer_manipulations as iman
import util_funcs as uf

In [2]:
def compute_hnf_props(hnf_mats, l_bp_p, l_p_po):
    num_hnf = len(hnf_mats);
    l_sig_p_mats = np.zeros((num_hnf, 3, 2));
    l_sig_po_mats = np.zeros((num_hnf, 3, 2));
    for hct1 in range(num_hnf):
        # print(hct1)
        l_sig_p = np.dot(l_bp_p, hnf_mats[hct1]);
        # l_sig1_p = lll_reduction_bpl_basis(l_sig_p, l_p_po);
        l_sig1_p = lt.reduce_po_lat(l_sig_p, Matrix(l_p_po), 1e-6)
        l_sig_p_mats[hct1] = l_sig1_p;
        l_sig_po = np.dot(l_p_po, l_sig1_p);
        l_sig_po_mats[hct1] = l_sig_po;
    return l_sig_p_mats, l_sig_po_mats;

Define the primitive unit cell

In [3]:
# get the lattice parameter of Al
l1 = gbl.Lattice('hp_id')
# Determine the primitive basis of fcc crystal structure
l_p_po = Matrix(l1.l_p_po)
l_po_p = l_p_po.inv()
## (h k l) indices of the fcc lattice (described in p)
h = 3
k = 4
l = 5

hkl_p = Matrix([h,k,l])

In [4]:
# hkl_p1
## l_bpb_p: Primitive Basis vectors of the boundary-plane (in p reference frame)
l_bpb_p = bpb.bp_basis(hkl_p)
l_bpb_p = Matrix(l_bpb_p.astype(int))
## l_bpb_p: Primitive Basis vectors of the boundary-plane (in po reference frame)
l_bpb_po = l_p_po*l_bpb_p

In [5]:
## Cut-off for area of the simulation box
lat_par = l1.lat_params['a']
r_cut = lat_par*4
A_cut = r_cut**2
## area_bpl: Area of the 2D-primitive-unit-cell
area_bpl = (l_bpb_po[:,0].cross(l_bpb_po[:,1])).norm()
# sig_num = np.ceil(A_cut/area_bpl)
sig_num = (A_cut/area_bpl).ceiling()

In [6]:
# here I choose the first HNF as the best one and apply no ll reduction
hnf_mats = ghi.sig_hnf_mats(sig_num)
# l_sig_p_mats, l_sig_po_mats = compute_hnf_props(hnf_mats, l_bpb_p, l_p_po)
# hnf_mats, l_bp_p, l_p_po
num_hnf = len(hnf_mats);
l_sig_p_mats = np.zeros((num_hnf, 3, 2));
l_sig_po_mats = np.zeros((num_hnf, 3, 2));
# for hct1 in range(num_hnf):
hct1 = 0
l_sig_p = Matrix(np.dot(np.array(l_bpb_p, dtype='int64'), hnf_mats[hct1]));
l_sig1_sig = uf.reduce_po_lat(l_sig_p, Matrix(l_p_po), 1e-2)
l_sig1_p = np.array(l_sig_p*l_sig1_sig, dtype='int64')
l_sig_p_mats[hct1] = l_sig1_p;
l_sig_po = np.dot(l_p_po, l_sig1_p);
l_sig_po_mats[hct1] = l_sig_po;
        
# l_bpbSig_p = np.dot(l_bpb_p, hnf_mats[0])
# l_bpbSig_p = Matrix(l_bpbSig_p.astype(int))
# l_bpbSig_po = l_p_po*l_bpbSig_p

In [7]:
l_sig_p

array([[ 1, -5],
       [-2,  0],
       [ 1,  3]])

In [12]:
l_bpb_p

Matrix([
[ 1, -2],
[-2, -1],
[ 1,  2]])

In [14]:
hnf_mats[hct1]

array([[1, 0],
       [0, 2]])

In [ ]:
l_po_go = uf.compute_orientation(l_bpbSig_po)
l2D_bpbSig_go = l_po_go*l_bpbSig_po
l2D_bpb_go = l_po_go*l_bpb_po
twoD_mat = l2D_bpb_go[:2,:]

second_atom = (np.array([l1.lat_params['a']/3, 2*l1.lat_params['a']/3, l1.lat_params['c']/2])).reshape(3,1)
shift = np.dot(l_po_go, second_atom)


In [ ]:
twoDSig_mat = l2D_bpbSig_go[:2,:]
r_cut1 = uf.compute_rCut(l_bpbSig_po)
## Create 2D periodic box
twoD_pts = uf.replicate_pts(twoD_mat, r_cut1)
twoD_pts = uf.replicate_pts(twoD_mat, r_cut1)
twoD_pts = uf.remove_periodic_overlaps(twoD_pts, twoDSig_mat)

twoDSig_mat_arr = np.array(twoDSig_mat,dtype='float')
a_vec = twoDSig_mat_arr[:,0]; b_vec = twoDSig_mat_arr[:,1]

zCut = 25*lat_par
avec = l_bpb_p[:,0]
bvec = l_bpb_p[:,1]

l_p2_p1 = uf.find_int_solns(avec, bvec)
l_p2_po1 = (l_p_po*l_p2_p1)
l_p2_go = (l_po_go*l_p2_po1)
tz_vec = np.array(l_p2_go[:,2], dtype='double')
tz_vec = np.reshape(tz_vec, (3,))

In [ ]:
################################################################################
## Translate 2D points in the Z-direction with zCut
num_rep = np.abs(int(np.ceil(zCut/tz_vec[2])))
num_2d = np.shape(twoD_pts)[0]
num_3d_pts = int((2*num_rep+1)*num_2d)
threeD_pts = np.zeros((num_3d_pts,3));

twoD_pts1 = np.hstack((twoD_pts, np.zeros((num_2d,1))));

for ct1 in np.arange(-num_rep, num_rep+1):
    ct2 = ct1 + num_rep;
    ind_st = (ct2)*num_2d;
    ind_stop = ind_st + num_2d;
    trans_vec = tz_vec*ct1;
    threeD_pts[ind_st:ind_stop, :] = twoD_pts1 + np.tile(trans_vec, (num_2d,1));

### Simulation Cell Box
### Following Ovito's convention
l_bpbSig_po_arr = np.array(l2D_bpbSig_go, dtype='double')


sim_cell = np.zeros((3,4))
sim_avec = l_bpbSig_po_arr[:,0]
sim_bvec = l_bpbSig_po_arr[:,1]

### Change this with inter-planar spacing
sim_cvec = np.array([0,0,2*zCut]); # sim_cvec = np.array([0,0,zCut]);
sim_orig = np.array([0,0,-zCut]); # sim_orig = np.array([0,0,0]);

sim_cell[:,0] = sim_avec
sim_cell[:,1] = sim_bvec
sim_cell[:,2] = sim_cvec
sim_cell[:,3] = sim_orig

box_vecs = sim_cell[:,0:3]
shift = shift.reshape(1,3)
# th_x = sim_cell[0,0] - sim_cell[0,3]
# th_y = sim_cell[1,1] - sim_cell[1,3]
th_z = sim_cell[2,2] + sim_cell[2,3]


threeD_pts_1 = threeD_pts + shift
threeD_pts_f = np.append(threeD_pts, threeD_pts_1,axis=0)
threeD_pts_f = threeD_pts_f[threeD_pts_f[:,2] <= th_z]
threeD_pts_f = threeD_pts_f[threeD_pts_f[:,2] >= -th_z]

threeD_pts_final = uf.wrap_cc(sim_cell, threeD_pts_f)


In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_2d_pts_box(twoD_pts, twoD_mat, orig):
    ################################################################################
    l1 = np.copy(twoD_mat);
    avec = l1[:,0]; bvec=l1[:,1];
    pts = np.zeros((4,2));
    pts[0,:] = orig;
    pts[1,:] = orig+avec; pts[2,:] = orig+avec+bvec; pts[3,:] = orig+bvec;
    tinds = [0,1,2,3,0];
    x1=pts[tinds,0];y1=pts[tinds,1];
    plt.plot(x1,y1);

    pts = np.copy(twoD_pts);
    x1=pts[:,0];y1=pts[:,1];
    plt.scatter(x1,y1);
    ################################################################################

def plot_3d_pts_box(fig, pts, tmat, sim_orig):
    ################################################################################
    ax = fig.add_subplot(111, projection='3d')

    box_pts = np.zeros((8,3));
    box_pts[1,:] = tmat[:,0]; box_pts[2,:] = tmat[:,0]+tmat[:,1];
    box_pts[3,:] = tmat[:,1];
    box_pts[4,:] = box_pts[0,:]+tmat[:,2];
    box_pts[5,:] = box_pts[1,:]+tmat[:,2];
    box_pts[6,:] = box_pts[2,:]+tmat[:,2];
    box_pts[7,:] = box_pts[3,:]+tmat[:,2];
    box_pts = box_pts + sim_orig;

    tinds = [0,1,2,3,0,4,5,6,7,4,0,1,5,6,2,3,7];
    x1 = box_pts[tinds,0]; y1 = box_pts[tinds,1]; z1 = box_pts[tinds,2];
    ax.plot(x1,y1,z1);

    x1=pts[:,0];y1=pts[:,1];z1=pts[:,2];
    ax.scatter(x1,y1,z1);

    ################################################################################

In [ ]:
fig1 = plt.figure()
plot_3d_pts_box(fig1, threeD_pts_final, sim_cell[:,0:3], sim_orig); plt.show();

In [ ]:
for tct1 in range(3):
    for tct2 in range(3):
        twoD_pts1 = 0*twoD_pts;
        orig = np.array([0,0])
        ct1 = tct1-1;
        ct2 = tct2-1;
        ct3 = 0; 
        twoD_pts1[:,ct3] = twoD_pts[:,ct3] + ct1*a_vec[ct3] + ct2*b_vec[ct3]
        ct3 = 1; 
        twoD_pts1[:,ct3] = twoD_pts[:,ct3] + ct1*a_vec[ct3] + ct2*b_vec[ct3]
        orig = ct1*a_vec + ct2*b_vec;
        plot_2d_pts_box(twoD_pts1, twoDSig_mat_arr, orig)
        # orig = ct1*a_vec + ct2*b_vec;
        # print(orig)

In [ ]:

def box_bound_func(sim_cell):
    origin_o = sim_cell[:, 3]
    xlo = origin_o[0]
    ylo = origin_o[1]
    zlo = origin_o[2]

    xhi = sim_cell[0, 0] + xlo
    xy = sim_cell[0, 1]
    yhi = sim_cell[1, 1] + ylo
    xz = sim_cell[0, 2]
    yz = sim_cell[1, 2]
    zhi = sim_cell[2, 2] + zlo

    if xy or xz or yz != 0:
        box_type = "prism"
    else:
        box_type = "block"

    xlo_bound = xlo + np.min(np.array([0, xy, xz, xy + xz]))
    xhi_bound = xhi + np.max(np.array([0, xy, xz, xy + xz]))
    ylo_bound = ylo + np.min(np.array([0, yz]))
    yhi_bound = yhi + np.max(np.array([0, yz]))
    zlo_bound = zlo
    zhi_bound = zhi

    if box_type == "block":
        box_bound = np.array([[xlo_bound, xhi_bound], [ylo_bound, yhi_bound], [zlo_bound, zhi_bound]])
    else:
        box_bound = np.array([[xlo_bound, xhi_bound, xy], [ylo_bound, yhi_bound,  xz], [zlo_bound, zhi_bound, yz]])
    return box_bound, box_type

def write_lammps_dump(filename0, box_bound, dump_lamp, box_type):
    """
    Function writes the lammps dump file.

    Parameters
    ------------
    filename0 :
        Name of the lammps dump file
    box_bound :
        The box bound needed to write lammps dump file which is 9 parameters: xlo, xhi, ylo, yhi,
        zlo, zhi, xy, xz, yz
    dump_lamp :
        A numpy nd.array having atom ID, atom type( 1 for upper grain and 2 for lower grain), x, y, z

    Returns
    ----------
    """
    p_x = box_bound[0, 1] - box_bound[0, 0]
    p_y = box_bound[1, 1] - box_bound[1, 0]
    p_z = box_bound[2, 1] - box_bound[2, 0]

    non_p_dir = np.argmax([p_x, p_y, p_z])

    num_atoms = np.shape(dump_lamp)[0]
    file = open(filename0, "w")
    file.write("ITEM: TIMESTEP\n")
    file.write("0\n")
    file.write("ITEM: NUMBER OF ATOMS\n")
    file.write(str(num_atoms) + "\n")
    # file.write("ITEM: BOX BOUNDS xy xz yz pp ff pp\n")
    if box_type == "prism":
        if non_p_dir == 0:
            file.write("ITEM: BOX BOUNDS xy xz yz ff pp pp\n")
        elif non_p_dir == 1:
            file.write("ITEM: BOX BOUNDS xy xz yz pp ff pp\n")
        else:
            file.write("ITEM: BOX BOUNDS xy xz yz pp pp ff\n")

    else:
        if non_p_dir == 0:
            file.write("ITEM: BOX BOUNDS ff pp pp\n")
        elif non_p_dir == 1:
            file.write("ITEM: BOX BOUNDS pp ff pp\n")
        else:
            file.write("ITEM: BOX BOUNDS pp pp ff\n")

    file.write(' '.join(map(str, box_bound[0])) + "\n")
    file.write(' '.join(map(str, box_bound[1])) + "\n")
    file.write(' '.join(map(str, box_bound[2])) + "\n")
    file.write("ITEM: ATOMS id x y z\n")
    file.close()
    mat = np.matrix(dump_lamp)
    with open(filename0, 'a') as f:
        for line in mat:
            np.savetxt(f, line, fmt='%d %.10f %.10f %.10f')

num_atoms = len(threeD_pts_final)
ID = np.arange(num_atoms).reshape(num_atoms, 1) + 1
dump_lamp = np.concatenate((ID, threeD_pts_final), axis=1)
box_bound, box_type = box_bound_func(sim_cell)
write_lammps_dump('./hcp.dump', box_bound, dump_lamp, box_type)